# Updating Variables in a dataframe. 
Creating scenarios requires updating variables  (columns in a dataframe). This can be done straight using pandas methods.<br>
But in some cases some useful modelflow functions can be handy. 

 %load_ext autoreload
 %autoreload 2


In [264]:
import pandas as pd

from modelclass import model 
model.widescreen()
model.scroll_off()

## Create dataframe to update: 

In [265]:
number_of_rows = 6 
df = pd.DataFrame([100.]*number_of_rows,index=[2020+v for v in range(number_of_rows)],columns=['A'])
df.loc[:,'B'.split() ]= 100.0
df

,A,B
2020,100.0,100.0
2021,100.0,100.0
2022,100.0,100.0
2023,100.0,100.0
2024,100.0,100.0
2025,100.0,100.0


## = Setting value equal to input

In [266]:
model.update(df,'''
# Same number of values as years
A = 42 44   / 2021 2022
''')

,A,B
2020,100.0,100.0
2021,42.0,100.0
2022,44.0,100.0
2023,100.0,100.0
2024,100.0,100.0
2025,100.0,100.0


## + adding input to previous value 

In [267]:
model.update(df,'''
# Or one number to all years in between start and end 
B  +  42 / 2022 2024
''')

,A,B
2020,100.0,100.0
2021,100.0,100.0
2022,100.0,142.0
2023,100.0,142.0
2024,100.0,142.0
2025,100.0,100.0


## * multipying  previous value by input 

In [268]:
model.update(df,'''
# Same number of values as years
A *  42 44   / 2021 2022
''')

,A,B
2020,100.0,100.0
2021,4200.0,100.0
2022,4400.0,100.0
2023,100.0,100.0
2024,100.0,100.0
2025,100.0,100.0


## % Increase by percent  

In [269]:
model.update(df,'''
# Same number of values as years
A %  42 44   / 2021 2022
same = 100 
same % 42 
''')

,A,B,SAME
2020,100.0,100.0,142.0
2021,142.0,100.0,142.0
2022,144.0,100.0,142.0
2023,100.0,100.0,142.0
2024,100.0,100.0,142.0
2025,100.0,100.0,142.0


## =GROWTH Set the percent growth rate   

In [270]:
res = model.update(df,'''
# Same number of values as years
A =GROWTH  1 5  / 2021 2022
# No years mean all years. The variable is not in the dataframe and set to 0 before update  
same = 100 /2020
same =growth 2 /2021 2025 
''')
print(f'Dataframe:\n{res}\n\nGrowth:\n{res.pct_change()*100}\n')

Dataframe:
           A      B        SAME
2020  100.00  100.0  100.000000
2021  101.00  100.0  102.000000
2022  106.05  100.0  104.040000
2023  100.00  100.0  106.120800
2024  100.00  100.0  108.243216
2025  100.00  100.0  110.408080

Growth:
             A    B  SAME
2020       NaN  NaN   NaN
2021  1.000000  0.0   2.0
2022  5.000000  0.0   2.0
2023 -5.704856  0.0   2.0
2024  0.000000  0.0   2.0
2025  0.000000  0.0   2.0



## +GROWTH increase the growth rate    

In [271]:
res =model.update(df,'''
# Same number of values as years
A =GROWTH  1   / 2021 2025
# now we add values  to the growth rate, so the growth rate should be 1 + the number 
a +growth   2 3 4 5 6 / 2021 2025
''')
print(f'Dataframe:\n{res}\n\nGrowth:\n{res.pct_change()*100}\n')

Dataframe:
               A      B
2020  100.000000  100.0
2021  103.000000  100.0
2022  107.120000  100.0
2023  112.476000  100.0
2024  119.224560  100.0
2025  127.570279  100.0

Growth:
        A    B
2020  NaN  NaN
2021  3.0  0.0
2022  4.0  0.0
2023  5.0  0.0
2024  6.0  0.0
2025  7.0  0.0



## ^ Set the absolute growth 

In [272]:
model.update(df,'''
# Same number of values as years
A ^  2 4   / 2021 2022
# cv number to all years in between start and end 

same = 100 /2020
same ^ 2 /2021 2025 
''')

,A,B,SAME
2020,100.0,100.0,100.0
2021,102.0,100.0,102.0
2022,106.0,100.0,104.0
2023,100.0,100.0,106.0
2024,100.0,100.0,108.0
2025,100.0,100.0,110.0


## Setting the time 
The time is set after the ```/``` as ```start end```
 - no start and end indicated the whole time range will be updated (**not good for growth updates**) 
 - only start then just this time is updated 
 - start and end, the update is carried on for the interval from start to end, including both 

## option  --keep_growth, keep the growth rate in the years (or whatever frequency) after the update
Sometime the user want to update a variable for a few years but keep the growth rate in the years after at the same level

In [273]:
# First make a dataframe with some growth rate 
res = model.update(df,'''
a =growth 1 2 3 4 5 /2021 2025
''') 
print(f'Dataframe:\n{res}\n\nGrowth:\n{res.pct_change()*100}\n')

Dataframe:
               A      B
2020  100.000000  100.0
2021  101.000000  100.0
2022  103.020000  100.0
2023  106.110600  100.0
2024  110.355024  100.0
2025  115.872775  100.0

Growth:
        A    B
2020  NaN  NaN
2021  1.0  0.0
2022  2.0  0.0
2023  3.0  0.0
2024  4.0  0.0
2025  5.0  0.0



In [274]:
# now update A in 2021 to 2023 to a new value, 
# and watch how values after 2023 are updated, so the gowth rate after the update is unchanged 
res = model.update(df,'''
a =growth 1 2 3 4 5 /2021 2025
a = 120 /2021 2023 --keep_growth
''')
print(f'Dataframe:\n{res}\n\nGrowth:\n{res.pct_change()*100}\n')

Dataframe:
           A      B
2020  100.00  100.0
2021  120.00  100.0
2022  120.00  100.0
2023  120.00  100.0
2024  124.80  100.0
2025  131.04  100.0

Growth:
         A    B
2020   NaN  NaN
2021  20.0  0.0
2022   0.0  0.0
2023   0.0  0.0
2024   4.0  0.0
2025   5.0  0.0



## .update(,,scale=\<number, default=1>) Scale the updates 
When creating scenarios consistent of several updates it can be useful to be able to create 
a number of scenarios based on one update but with different scale. For instance scale=0 is the baseline wile scale=0.5 is a scenario half 
the severity.  

In [275]:
print(f'input dataframe: \n{df}\n\n')
for serverity in [0,0.5,1]: 
    # First make a dataframe with some growth rate 
    res = model.update(df,'''
    a =growth 1 2 3 4 5 /2021 2025
    b + 10
    ''',scale=serverity)
    print(f'{serverity=}\nDataframe:\n{res}\n\nGrowth:\n{res.pct_change()*100}\n\n')

input dataframe: 
          A      B
2020  100.0  100.0
2021  100.0  100.0
2022  100.0  100.0
2023  100.0  100.0
2024  100.0  100.0
2025  100.0  100.0


serverity=0
Dataframe:
          A      B
2020  100.0  100.0
2021  100.0  100.0
2022  100.0  100.0
2023  100.0  100.0
2024  100.0  100.0
2025  100.0  100.0

Growth:
        A    B
2020  NaN  NaN
2021  0.0  0.0
2022  0.0  0.0
2023  0.0  0.0
2024  0.0  0.0
2025  0.0  0.0


serverity=0.5
Dataframe:
               A      B
2020  100.000000  105.0
2021  100.500000  105.0
2022  101.505000  105.0
2023  103.027575  105.0
2024  105.088126  105.0
2025  107.715330  105.0

Growth:
        A    B
2020  NaN  NaN
2021  0.5  0.0
2022  1.0  0.0
2023  1.5  0.0
2024  2.0  0.0
2025  2.5  0.0


serverity=1
Dataframe:
               A      B
2020  100.000000  110.0
2021  101.000000  110.0
2022  103.020000  110.0
2023  106.110600  110.0
2024  110.355024  110.0
2025  115.872775  110.0

Growth:
        A    B
2020  NaN  NaN
2021  1.0  0.0
2022  2.0  0.0
2023  

## .update(,,lprint=True ) prints the before and after update  
When creating scenarios consistent of several updates it can be useful to be able to create 
a number of scenarios based on one update but with different scale. For instance scale=0 is the baseline wile scale=0.5 is a scenario half 
the severity.  

In [276]:
model.update(df,'''
# Same number of values as years
A *  42 44   / 2021 2022
''',lprint=1)

Update * [42.0, 44.0] 2021 2022
A                    Before                After                 Diff
2021               100.0000            4200.0000            4100.0000
2022               100.0000            4400.0000            4300.0000


,A,B
2020,100.0,100.0
2021,4200.0,100.0
2022,4400.0,100.0
2023,100.0,100.0
2024,100.0,100.0
2025,100.0,100.0


## The call 

def update(indf, basis, lprint=False,scale = 1.0,create=True,keep_growth=False ):

    Args:
            indf (TYPE): input dataframe.
            basis (TYPE): lines with variable updates look below.
            lprint (TYPE, optional): if True each update is printed  Defaults to False.
            scale (TYPE, optional): A multiplier used on all update input . Defaults to 1.0.
            create (TYPE, optional): Creates a variables if not in the dataframe . Defaults to True.

        Returns:
            df (TYPE): the updated dataframe .
            
        An update line looks like this:     
            
        <var> <=|+|*|%|=growth|+growth|^> <value>... [/ [[start] end] [--keep_growth_rate]]       
